# Context View

## 0. Initialization

### 0.1. Importing libraries and setting global variables

In [1]:
import esm
import os
import trimesh as tm 
import ladybug as lb
from honeybee_plus.radiance.recipe.solaraccess.gridbased import SolarAccessGridBased
from honeybee_plus.radiance.analysisgrid import AnalysisGrid

cwd = os.getcwd()
src = os.path.dirname(cwd)

Path to radiance is set to: /usr/local/radiance
Path to perl is set to: /usr/bin


### 0.2. Importing geometries and assigning materials

In [2]:
mesh_files = ["test_room_walls_02.obj", "test_room_fens_02.obj", "test_context_01_buildings.obj", "test_context_01_ground.obj", "test_context_01_vegetation.obj"]
srf_types = [0.0, 0.5, 6.0, 6.0, 6.0]
srf_names = ["wall", "window", "context_buildings", "context_ground", "context_vegetation"]
srf_mats = [esm.material_plastic, esm.material_glass, esm.material_plastic, esm.material_plastic, esm.material_plastic]

hb_surfaces = []
for mesh_file, srf_type, srf_name, srf_mat in zip(mesh_files, srf_types, srf_names, srf_mats):
    # file path
    mesh_filepath = os.path.join(src, "data", mesh_file)
    # load mesh
    mesh = tm.load(mesh_filepath)
    # create surface
    hb_surf = esm.mesh_to_hbsurface(mesh.faces, mesh.vertices, srf_type, srf_name, srf_mat)
    hb_surfaces.extend(hb_surf)

faces have mixed data, using slow fallback!
face lines are weird: 1 6 3 4 5 2
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!


### 0.3. Prepare Analysis Information

In [3]:
# generate grid of test points
test_pts = [(1, 1, 1),(6, 1, 1),(1, 6, 1),(1, 1, 6)]
test_vec = [(0, 0, 1),(0, 0, 1),(0, 0, 1),(0, 0, 1)]
analysis_grid = AnalysisGrid.from_points_and_vectors(test_pts, test_vec, 'test_grid')

## 1. Simulation

### 1.1. Create Recipe

In [4]:
import esm

# put the recipe together
rp = esm.ContextViewGridBased(analysis_grids=(analysis_grid,), hb_objects=hb_surfaces)

Found 35213 opaque surfaces.
Found 1 fenestration surfaces.
Found 0 window-groups.


### 1.2. Create Batch Files

In [5]:
# write simulation to folder
batch_file = rp.write(
    target_folder='.', project_name='context_view_cleaned')

Writing recipe contents to: ./context_view_cleaned/gridbased
Changed view_h_size and view_v_size to 180 for fisheye view type.


AttributeError: irradiance_calc is not a valid parameter for RtraceParameters. Failed to set irradiance_calc to True.
Use `.parameters` method to see the list of available parameters.
Use one of the add*Parameters methods to add a new parameter.

In [15]:
import honeybee_plus

rp_rpict = honeybee_plus.radiance.parameters.rpict.RpictParameters(0)
honeybee_plus.radiance.command.rpict.Rpict()
print(type(rp_rpict.))

<class 'honeybee_plus.radiance.parameters.rpict.RpictParameters'>


### 1.3. Run the simulation

In [6]:
# run the simulation
rp.run(batch_file, debug=False)

True

## 2. Results

In [7]:
rp._result_files

'./context_view_cleaned/gridbased/rtrace_res.txt'

In [8]:
result = esm.parse_results(rp)
print(result)

EmptyDataError: No columns to parse from file